# Simple Image Classification Models for the CIFAR-10 dataset using TensorFlow

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import time
from datetime import datetime
import os.path
import data_helpers
import two_layer_fc
import sys


## Model parameters as external flags

In [2]:
flags = tf.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.001, 'Learning rate for the training.')
flags.DEFINE_integer('max_steps', 2000, 'Number of steps to run trainer.')
flags.DEFINE_integer('hidden1', 120, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('batch_size', 400,
  'Batch size. Must divide dataset sizes without remainder.')
flags.DEFINE_string('train_dir', 'tf_logs',
  'Directory to put the training data.')
flags.DEFINE_float('reg_constant', 0.1, 'Regularization constant.')
tf.app.flags.DEFINE_string('f', '', 'kernel')
FLAGS(sys.argv)

print('\nParameters:')
for attr, value in sorted(FLAGS.__flags.items()):
  print('{} = {}'.format(attr, value))
print()

IMAGE_PIXELS = 3072
CLASSES = 10

beginTime = time.time()


Parameters:
batch_size = <absl.flags._flag.Flag object at 0x000002CBF70240B8>
f = <absl.flags._flag.Flag object at 0x000002CBF70241D0>
hidden1 = <absl.flags._flag.Flag object at 0x000002CBF7027EF0>
learning_rate = <absl.flags._flag.Flag object at 0x000002CBF7027EB8>
max_steps = <absl.flags._flag.Flag object at 0x000002CBF7027F60>
reg_constant = <absl.flags._flag.Flag object at 0x000002CBF7024128>
train_dir = <absl.flags._flag.Flag object at 0x000002CBF7024080>



## Put logs for each run in separate directory

In [3]:
logdir = FLAGS.train_dir + '/' + datetime.now().strftime('%Y%m%d-%H%M%S') + '/'

In [4]:
# Uncommenting these lines removes randomness
# You'll get exactly the same result on each run
# np.random.seed(1)
# tf.set_random_seed(1)

## Load CIFAR-10 data

In [5]:
data_sets = data_helpers.load_data()

## Prepare the Tensorflow graph

## Define input placeholders

In [6]:
images_placeholder = tf.placeholder(tf.float32, shape=[None, IMAGE_PIXELS],
  name='images')
labels_placeholder = tf.placeholder(tf.int64, shape=[None], name='image-labels')

## Operation for the classifier's result

In [7]:
logits = two_layer_fc.inference(images_placeholder, IMAGE_PIXELS,
  FLAGS.hidden1, CLASSES, reg_constant=FLAGS.reg_constant)

## Operation for the loss function

In [8]:
loss = two_layer_fc.loss(logits, labels_placeholder)

## Operation for the training step

In [9]:
train_step = two_layer_fc.training(loss, FLAGS.learning_rate)

## Operation calculating the accuracy of our predictions

In [10]:
accuracy = two_layer_fc.evaluation(logits, labels_placeholder)

## Operation merging summary data for TensorBoard

In [11]:
summary = tf.summary.merge_all()

## Define saver to save model state at checkpoints


In [12]:
saver = tf.train.Saver()

## Run the TensorFlow graph

In [13]:
with tf.Session() as sess:
  # Initialize variables and create summary-writer
  sess.run(tf.global_variables_initializer())
  summary_writer = tf.summary.FileWriter(logdir, sess.graph)

  # Generate input data batches
  zipped_data = zip(data_sets['images_train'], data_sets['labels_train'])
  batches = data_helpers.gen_batch(list(zipped_data), FLAGS.batch_size,
    FLAGS.max_steps)

  for i in range(FLAGS.max_steps):

    # Get next input data batch
    batch = next(batches)
    images_batch, labels_batch = zip(*batch)
    feed_dict = {
      images_placeholder: images_batch,
      labels_placeholder: labels_batch
    }

    # Periodically print out the model's current accuracy
    if i % 100 == 0:
      train_accuracy = sess.run(accuracy, feed_dict=feed_dict)
      print('Step {:d}, training accuracy {:g}'.format(i, train_accuracy))
      summary_str = sess.run(summary, feed_dict=feed_dict)
      summary_writer.add_summary(summary_str, i)

    # Perform a single training step
    sess.run([train_step, loss], feed_dict=feed_dict)

    # Periodically save checkpoint
    if (i + 1) % 1000 == 0:
      checkpoint_file = os.path.join(FLAGS.train_dir, 'checkpoint')
      saver.save(sess, checkpoint_file, global_step=i)
      print('Saved checkpoint')

  # After finishing the training, evaluate on the test set
  test_accuracy = sess.run(accuracy, feed_dict={
    images_placeholder: data_sets['images_test'],
    labels_placeholder: data_sets['labels_test']})
  print('Test accuracy {:g}'.format(test_accuracy))

endTime = time.time()
print('Total time: {:5.2f}s'.format(endTime - beginTime))

Step 0, training accuracy 0.13
Step 100, training accuracy 0.3125
Step 200, training accuracy 0.335
Step 300, training accuracy 0.4075
Step 400, training accuracy 0.4225
Step 500, training accuracy 0.425
Step 600, training accuracy 0.4475
Step 700, training accuracy 0.495
Step 800, training accuracy 0.525
Step 900, training accuracy 0.4675
Saved checkpoint
Step 1000, training accuracy 0.5
Step 1100, training accuracy 0.5125
Step 1200, training accuracy 0.5175
Step 1300, training accuracy 0.465
Step 1400, training accuracy 0.51
Step 1500, training accuracy 0.5475
Step 1600, training accuracy 0.525
Step 1700, training accuracy 0.5225
Step 1800, training accuracy 0.575
Step 1900, training accuracy 0.5275
Saved checkpoint
Test accuracy 0.4526
Total time: 69.60s
